In [1]:
import csv,nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import  pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer


In [8]:

with open('D:\Development\Minor Project-Ontology Based\Sample_Doctor_Dataset\CSV Files\doid_updated.csv',
          'r') as csvFile:
    reader = csv.reader(csvFile)
    data = list(reader)
csvFile.close()

In [17]:
class feature_set_words:
    def __init__(self, list_of_indices):
        self.list_of_indices = list_of_indices

    def building_dataset(self,words,output_class):
        dataset=[]
        temp={}
        for j in range(0, len(data)):
            temp={}
            tokens = (self.tokenizing([data[j][1]]))
            for i in range(0, len(tokens)):
                if tokens[i] in words:
                    temp[tokens[i]]=True
                    break
                else:
                    continue
            if data[j][i]==output_class:
                dataset.append((temp,output_class))
            else:
                dataset.append((temp, "Not "+output_class))
        return dataset
    def changing_for_count_vectorization(self,words):
        temp = []
        sent = ''
        for i in words:
            for j in i:
                sent = sent + ' ' + str(j)
            temp.append(sent.strip())
            sent = ' '
        return temp

    def feature_set_formation_for_incorrect_words(self):
        final_words_for_count_vectorization = []
        for i in range(1, len(data)):
            if i in self.list_of_indices:
                continue
            else:
                word_arr = self.tokenizing([data[i][1]])
                stopwords_remove = self.stop_words_removal(word_arr)
                parts_of_speech = self.parts_of_speech(data[i][1])
                words = self.lemmatization(parts_of_speech, stopwords_remove)
                final_words_for_count_vectorization.append(words)

        temp = self.changing_for_count_vectorization(final_words_for_count_vectorization)
        count_vec = CountVectorizer(max_features=10)
        x_train_features = count_vec.fit_transform(temp[:])
        # print(len(count_vec.get_feature_names()))
        print((count_vec.get_feature_names()))
        # print(x_train_features.todense())
        return count_vec.get_feature_names()

    def feature_set_formation_for_correct_words(self):
        final_words_for_count_vectorization=[]
        for i in range(1,len(self.list_of_indices)):
            word_arr = self.tokenizing([data[i][1]])
            stopwords_remove = self.stop_words_removal(word_arr)
            parts_of_speech = self.parts_of_speech(data[i][1])
            words = self.lemmatization(parts_of_speech, stopwords_remove)
            final_words_for_count_vectorization.append(words)

        # print(final_words_for_count_vectorization)
        temp=self.changing_for_count_vectorization(final_words_for_count_vectorization)

        count_vec = CountVectorizer(max_features=10)
        x_train_features = count_vec.fit_transform(temp[:25])
        # print(len(count_vec.get_feature_names()))
        print((count_vec.get_feature_names()))
        # print(x_train_features.todense())
        return count_vec.get_feature_names()
    def freq(self, words):
        dict = {
        }
        for i in words:
            dict[i] = words.count(i)
        # print("Dictionary after stopwords removal(no lemmatization)")
        # print(dict)
        return dict

    def pos_to_wordnet(self, pos_tag):
        if pos_tag.startswith('J'):
            return wordnet.ADJ
        elif pos_tag.startswith('N'):
            return wordnet.NOUN
        elif pos_tag.startswith('V'):
            return wordnet.VERB
        # elif pos_tag.startswith('M'):
        #     return wordnet.MODAL
        # elif pos_tag.startswith('R'):
        #     return wordnet.ADVERB
        else:
            return wordnet.NOUN

    def lemmatization(self, part_of_speech, clean_words):
        lemmatized = []
        lemmatizer = WordNetLemmatizer()
        # print pos[2][0]
        # print clean_words[1]
        for i in range(0, len(part_of_speech)):
            for j in range(0, len(clean_words)):
                if part_of_speech[i][0].lower() == clean_words[j]:
                    # print part_of_speech[i][0], clean_words[j], part_of_speech[i][1]
                    lemmatized.append(
                        (lemmatizer.lemmatize(clean_words[j], pos=self.pos_to_wordnet(part_of_speech[i][1]))).lower())
                    break
                else:
                    continue
        # print("After Lemmatization No. of Words:", len(list(set(lemmatized))))
        return list(set(lemmatized))

    def parts_of_speech(self,definition):
        parts_of_speech = pos_tag(word_tokenize(definition))  # +' '+self.synonym+ ' ' + self.name))
        # print(parts_of_speech)
        return parts_of_speech

    def freq(self, words):
        dict = {
        }
        for i in words:
            dict[i] = words.count(i)
        # print("Dictionary after stopwords removal(no lemmatization)")
        # print(dict)
        return dict

    def stemming(self, cleaning_words):
        '''
        The process of stemmimng is very dumb. Not always give reslutant output. The information passed through it might result in bad output.
        Stemming is the process of finding the root word of the given word.
        Prefer Lemmatization over it.
        '''
        ps = PorterStemmer()
        # stem_words = ['play', 'playing', 'played', 'player', "happy", 'happier']
        stemmed_words = [ps.stem(w) for w in cleaning_words]
        # print(stem_words)
        return stemmed_words

    def stop_words_removal(self, word_arr):
        # Stopwords and Punctuations.........................................
        stop_words = stopwords.words('english')
        # Stop Words are present of different languages, for papers of different languages.
        # Cleaning words(removing stopwords and punctuations
        # print(stop_words)
        import string
        punctuations = list(string.punctuation)
        # print(string.punctuation)
        stop_words += punctuations
        # print(len(stop_words))
        file_data = []
        with open('D:\Development\Minor Project-Ontology Based\Sample_Doctor_Dataset\All Notebooks\stopwords', 'r') as f:
            for line in f:
                for word in line.split():
                    file_data.append(word)
        stop_words += file_data
        stop_words = set(stop_words)
        stop_words = list(stop_words)
        new_word_arr = []
        for i in word_arr:
            new_word_arr.append(i.lower())
        clean_words = [w for w in new_word_arr if not w in stop_words]
        # print((stop_words))
        # print("After cleaning the stopwords, no of words:", len(clean_words))
        return clean_words

    def tokenizing(self, parameters):
        # Tokenising..............................................
        data = '';
        for i in parameters:
            data += i + ' '
        data = data.strip()
        # print(data)
        data = data.split('_')
        # print(data)
        sentence = ''
        for i in data:
            sentence += i
        word_arr = word_tokenize(sentence.lower())
        #  Here lower case is done to remove more stopwords, but some information is lost
        # print word_arr
        return word_arr


In [18]:
def encoding_y_train():
    encoding_output = {}
    c = 1
    for i in range(1, len(data)):
        if data[i][3] in encoding_output.keys():
            continue
        else:
            encoding_output[data[i][3]] = c
            c = c + 1
    # print(len(encoding_output))
    # print(encoding_output)
    y_train_test = []
    for i in range(1, len(data)):
        y_train_test.append(encoding_output[data[i][3]])
    # print(y_train_test)
    return y_train_test, encoding_output


In [19]:
def creating_feature_set(output_class):
    y_train_test,encoding_output=encoding_y_train()
    # print(encoding_output)
    temp_data=[]
    # print(encoding_output['drug allergy'])
    for i in encoding_output:
        if i=='drug allergy':
            for j in range(0,len(data)):
                if i==data[j][3]:
                    temp_data.append(j)
                else:
                    continue
            print(temp_data)
        else:
            continue
    feature_set=feature_set_words(temp_data)
    words_correct=feature_set.feature_set_formation_for_correct_words()
    words_incorrect=feature_set.feature_set_formation_for_incorrect_words()
    incorrect_rows=[]
    words_incorrect=set(words_incorrect)
    for j in range(0,len(data)):
        tokens=(feature_set.tokenizing([data[j][1]]))
        for i in range(0,len(tokens)):
            if tokens[i] in words_incorrect:
                incorrect_rows.append(j)
                break
            else:
                continue

    print(len(incorrect_rows))
    dataset=feature_set.building_dataset(list(words_incorrect)+words_correct,'drug allergy')
    print(dataset)

    data_input = ['patent blue V allergy',
              'A drug allergy that has_allergic_trigger patent blue V.',
              'allergic contact dermatitis to DNP']


In [20]:
creating_feature_set('abc')


[5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 29, 30, 31, 32, 34, 35, 37, 38, 39, 67, 68, 72, 73, 78, 79, 80, 82, 84, 1540]


['abacavir', 'allergy', 'beta', 'cephalosporin', 'disease', 'drug', 'lactam', 'lidocaine', 'line', 'lymphatic']
